## Xarray engine: auxiliary coordinates

In [1]:
import earthkit.data as ekd

### Basic examples

In [2]:
ds_fl = ekd.from_source("sample", "pl.grib")
ds_fl.ls().head()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,t,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
1,ecmf,r,isobaricInhPa,700,20240603,0,0,fc,0,regular_ll
2,ecmf,t,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
3,ecmf,r,isobaricInhPa,500,20240603,0,0,fc,0,regular_ll
4,ecmf,t,isobaricInhPa,700,20240603,0,6,fc,0,regular_ll


In [3]:
ds = ds_fl.to_xarray(
    profile="grib",
    aux_coords={"expver": ("expver", "forecast_reference_time")}, 
)
ds.load()

<xarray.Dataset> Size: 176kB
Dimensions:                  (forecast_reference_time: 4, step: 2, level: 2,
                              latitude: 19, longitude: 36)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * level                    (level) int64 16B 500 700
    expver                   (forecast_reference_time) <U4 64B '0001' ... '0001'
  * latitude                 (latitude) float64 152B 90.0 80.0 ... -80.0 -90.0
  * longitude                (longitude) float64 288B 0.0 10.0 ... 340.0 350.0
Data variables:
    r                        (forecast_reference_time, step, level, latitude, longitude) float64 88kB ...
    t                        (forecast_reference_time, step, level, latitude, longitude) float64 88kB ...
Attributes:
    Conventions:  CF-1.8
    institution:  ECMWF

In [4]:
ds2 = ds_fl.to_xarray(
    profile="grib",
    remapping={"centre_expver": "{centre}_{expver}"}, 
    aux_coords={"centre_and_expver": ("centre_expver", ("forecast_reference_time", "step"))}, 
)
ds2.load()

<xarray.Dataset> Size: 176kB
Dimensions:                  (forecast_reference_time: 4, step: 2, level: 2,
                              latitude: 19, longitude: 36)
Coordinates:
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * level                    (level) int64 16B 500 700
    centre_and_expver        (forecast_reference_time, step) <U9 288B 'ecmf_0...
  * latitude                 (latitude) float64 152B 90.0 80.0 ... -80.0 -90.0
  * longitude                (longitude) float64 288B 0.0 10.0 ... 340.0 350.0
Data variables:
    r                        (forecast_reference_time, step, level, latitude, longitude) float64 88kB ...
    t                        (forecast_reference_time, step, level, latitude, longitude) float64 88kB ...
Attributes:
    Conventions:  CF-1.8
    institution:  ECMWF

The feature of declaring auxiliary coordinates works also with the mono_variable setting

In [5]:
ds3 = ds_fl.to_xarray(
    profile="grib",
    fixed_dims=["param", "forecast_reference_time", "step", "level"], 
    mono_variable=True, 
    remapping={"centre_expver": "{centre}_{expver}"}, 
    aux_coords={"centre_and_expver": ("centre_expver", ("forecast_reference_time", "step"))}, 
)
ds3.load()

<xarray.Dataset> Size: 176kB
Dimensions:                  (param: 2, forecast_reference_time: 4, step: 2,
                              level: 2, latitude: 19, longitude: 36)
Coordinates:
  * param                    (param) <U1 8B 'r' 't'
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 32B 202...
  * step                     (step) timedelta64[ns] 16B 00:00:00 06:00:00
  * level                    (level) int64 16B 500 700
    centre_and_expver        (forecast_reference_time, step) <U9 288B 'ecmf_0...
  * latitude                 (latitude) float64 152B 90.0 80.0 ... -80.0 -90.0
  * longitude                (longitude) float64 288B 0.0 10.0 ... 340.0 350.0
Data variables:
    data                     (param, forecast_reference_time, step, level, latitude, longitude) float64 175kB ...
Attributes:
    Conventions:  CF-1.8
    institution:  ECMWF

### Quantiles in a probabilistic forecast

Let us now consider a probabilistic forecast of 2-metre temperature.

In [6]:
ds_fl2 = ekd.from_source("sample", "quantiles_pd.grib")

In this dataset, the fields are indexed by the metadata key ``"quantile"``, which is in turn composed of ``"number"`` and ``"numberOfForecastsInEnsemble"``

In [7]:
ds_fl2.ls(keys=[
    "shortName", 
    "dataDate", 
    "dataTime", 
    "stepRange", 
    "dataType", 
    "quantile", 
    "number", 
    "numberOfForecastsInEnsemble"
])

,shortName,dataDate,dataTime,stepRange,dataType,quantile,number,numberOfForecastsInEnsemble
0,2tp,20251209,0,0-168,pd,1:3,1,3
1,2tp,20251209,0,0-168,pd,1:5,1,5
2,2tp,20251209,0,0-168,pd,1:10,1,10
3,2tp,20251209,0,0-168,pd,2:3,2,3
4,2tp,20251209,0,0-168,pd,2:5,2,5
5,2tp,20251209,0,0-168,pd,2:10,2,10
6,2tp,20251209,0,0-168,pd,3:3,3,3
7,2tp,20251209,0,0-168,pd,3:5,3,5
8,2tp,20251209,0,0-168,pd,3:10,3,10
9,2tp,20251209,0,0-168,pd,4:5,4,5


Note that, in this context, the usual meaning of the metadata key ``"number"`` (and the related ``"numberOfForecastsInEnsemble"``) is overridden by ``"quantile"``. As a result, the ensemble dimension normally derived from ``"number"`` is no longer applicable.

For this reason, we must:
- declare ``"quantile"`` as an extra dimension, and
- remove the predefined ensemble dimension ``"number"``, since it would otherwise conflict with the ``"quantile"`` dimension.

Still, it might be useful to keep the information carried by ``"number"`` and ``"numberOfForecastsInEnsemble"`` is auxiliary coordinates.

In [8]:
ds4 = ds_fl2.to_xarray(
    profile="grib", 
    squeeze=False, 
    extra_dims="quantile", 
    drop_dims="number", 
    add_earthkit_attrs=False, 
    aux_coords={
        "quantile_rank": ("number", "quantile"), 
        "nquantiles": ("numberOfForecastsInEnsemble", "quantile")
    }
)
ds4.load()

<xarray.Dataset> Size: 13kB
Dimensions:                  (quantile: 18, forecast_reference_time: 1,
                              step: 1, level: 1, level_type: 1, latitude: 7,
                              longitude: 12)
Coordinates:
  * quantile                 (quantile) <U5 360B '10:10' '1:10' ... '9:10'
  * forecast_reference_time  (forecast_reference_time) datetime64[ns] 8B 2025...
  * step                     (step) timedelta64[ns] 8B 7 days
  * level                    (level) int64 8B 0
  * level_type               (level_type) <U7 28B 'surface'
    quantile_rank            (quantile) int64 144B 10 1 1 1 2 2 ... 5 5 6 7 8 9
    nquantiles               (quantile) int64 144B 10 10 3 5 10 ... 10 10 10 10
  * latitude                 (latitude) float64 56B 90.0 60.0 ... -60.0 -90.0
  * longitude                (longitude) float64 96B 0.0 30.0 ... 300.0 330.0
Data variables:
    2tp                      (quantile, forecast_reference_time, step, level, level_type, latitude, longitude) float64 12kB ...
Attributes:
    Conventions:  CF-1.8
    institution:  ECMWF